In [125]:
import numpy as np
from scipy.optimize import linear_sum_assignment
from scipy.stats import unitary_group
import scipy
import qiskit

In [2]:
n_anchors = 6
n_samples = 5
state_dim = 2
data = np.random.rand(n_anchors, n_samples, state_dim)

In [3]:
def minimize_permutation(data_A,data_B, axis):
    
    data_A = np.moveaxis(data_A, axis,0)
    data_B = np.moveaxis(data_B, axis,0)
    
    print(data_A.shape, data_B.shape)
    
    cost_matrix = np.zeros(shape=(data_A.shape[0], data_B.shape[0]))
    for i in range(len(cost_matrix)):
        for j in range(len(cost_matrix)):
            cost_matrix[i,j] = np.linalg.norm(data_B[i] - data_A[j])**2 ## Row for B and Column for A
        
    row_ind, col_ind = linear_sum_assignment(cost_matrix) 
    
    #print(cost_matrix.shape)
    
    permuted_data_A = data_A[col_ind]
    print(data_A.shape, data_B.shape, permuted_data_A.shape)

    
    print("Permutation difference", np.linalg.norm(data_A - permuted_data_A))
    print("Minimized cost", np.linalg.norm(data_B - permuted_data_A))
    
    data_A = np.moveaxis(data_A, 0, axis)
    permuted_data_A = np.moveaxis(permuted_data_A, 0, axis)
    data_B = np.moveaxis(data_B, 0, axis)
    print("Permutation difference", np.linalg.norm(data_A - permuted_data_A))
    print("Minimized cost", np.linalg.norm(data_B - permuted_data_A))
    
    return permuted_data_A, data_B

In [4]:
data_A = data
data_B = data[:,np.random.permutation(data.shape[1]),:]

In [5]:
minimize_permutation(data_A,data_B,1)

(5, 6, 2) (5, 6, 2)
(5, 6, 2) (5, 6, 2) (5, 6, 2)
Permutation difference 0.0
Minimized cost 0.0
Permutation difference 0.0
Minimized cost 0.0


(array([[[1.43558906e-01, 4.49921473e-01],
         [8.14830218e-01, 4.73444395e-01],
         [6.87019700e-01, 4.96014106e-01],
         [7.63618619e-01, 4.18770821e-01],
         [2.51005657e-02, 4.36022959e-01]],
 
        [[7.98650449e-01, 7.08430176e-01],
         [9.72523310e-01, 1.76899396e-01],
         [5.80533337e-01, 6.16868480e-01],
         [1.85261224e-01, 3.69118240e-01],
         [1.70810041e-01, 4.12695136e-01]],
 
        [[4.76831242e-01, 2.31648479e-01],
         [6.17281869e-01, 6.41142115e-01],
         [2.82770075e-01, 9.87547720e-01],
         [3.97625595e-02, 1.09776995e-02],
         [7.33092975e-01, 1.33117010e-01]],
 
        [[1.16013315e-01, 5.95788923e-02],
         [7.23491540e-02, 7.49846926e-01],
         [3.79898295e-01, 5.72601466e-01],
         [7.71791423e-01, 2.59829591e-01],
         [5.59993657e-01, 6.19960241e-02]],
 
        [[5.26518267e-01, 2.80813187e-01],
         [8.78644066e-04, 5.30022731e-02],
         [7.24836808e-01, 7.29268484e-01],

In [6]:
def minimize_unitary(data_A, data_B):
    """
    Min |A x Omega - B|^2, where Omega is a unitary matrix
    """
    A = data_A.reshape(-1,data_A.shape[2])
    B = data_B.reshape(-1,data_B.shape[2])
    
    M = A.conj().T @ B
    U,Sigma,V_d = np.linalg.svd(M)
    rank = sum(Sigma > 10e-6) # =2

    W = np.eye(U.shape[1], dtype=np.complex_)
    if U.shape[1] - rank > 1:
        W[rank:, rank:] = random_unitary(U.shape[1] - rank).data
    Omega = U @ W @ V_d # == U @ V_d
    
    transformed_A = A @ Omega
    
    #print(np.linalg.norm(transformed_A - B)**2)
    #print(transformed_A.shape, data_A.shape)
    #print(Omega)
    
    transformed_data_A = transformed_A.reshape(data_A.shape[0],data_A.shape[1],-1)
    print("Unitary transform difference", np.linalg.norm(data_A - transformed_data_A))
    print("Minimized cost", np.linalg.norm(data_B - transformed_data_A))
    return transformed_data_A, data_B

In [7]:
data_A = data

U = unitary_group.rvs(2)
unitary_tf = lambda x: U@x

data_B = np.apply_along_axis(unitary_tf, 2, data_A)

minimize_unitary(data_A, data_B)

Unitary transform difference 6.68506414777382
Minimized cost 1.6185992611541387e-15


(array([[[-0.31632832+0.21165412j, -0.17402019-0.21884787j],
         [-0.22202675+0.59537092j,  0.09795609-0.68901384j],
         [-0.26044012+0.53018992j,  0.03205521-0.60668618j],
         [-0.18922572+0.5506939j ,  0.10480407-0.639084j  ],
         [-0.32559704+0.14110081j, -0.21692835-0.13328645j]],
 
        [[-0.40246142+0.65473615j, -0.03153271-0.74031543j],
         [ 0.0285994 +0.59752249j,  0.31949665-0.71914242j],
         [-0.36963081+0.50560999j, -0.07610995-0.56522965j],
         [-0.2482486 +0.21152949j, -0.11419053-0.22618229j],
         [-0.28362139+0.2161098j , -0.14305242-0.22778238j]],
 
        [[-0.09558428+0.33517911j,  0.08102055-0.39113084j],
         [-0.38185381+0.53331199j, -0.07321686-0.59708073j],
         [-0.69971866+0.44641481j, -0.39565395-0.45801198j],
         [-0.00166324+0.02552116j,  0.01110844-0.03039864j],
         [ 0.02173282+0.45034828j,  0.24095856-0.54203185j]],
 
        [[-0.02569018+0.08248684j,  0.01803254-0.09601821j],
         [-0.55

In [8]:
data

array([[[1.43558906e-01, 4.49921473e-01],
        [8.14830218e-01, 4.73444395e-01],
        [6.87019700e-01, 4.96014106e-01],
        [7.63618619e-01, 4.18770821e-01],
        [2.51005657e-02, 4.36022959e-01]],

       [[7.98650449e-01, 7.08430176e-01],
        [9.72523310e-01, 1.76899396e-01],
        [5.80533337e-01, 6.16868480e-01],
        [1.85261224e-01, 3.69118240e-01],
        [1.70810041e-01, 4.12695136e-01]],

       [[4.76831242e-01, 2.31648479e-01],
        [6.17281869e-01, 6.41142115e-01],
        [2.82770075e-01, 9.87547720e-01],
        [3.97625595e-02, 1.09776995e-02],
        [7.33092975e-01, 1.33117010e-01]],

       [[1.16013315e-01, 5.95788923e-02],
        [7.23491540e-02, 7.49846926e-01],
        [3.79898295e-01, 5.72601466e-01],
        [7.71791423e-01, 2.59829591e-01],
        [5.59993657e-01, 6.19960241e-02]],

       [[5.26518267e-01, 2.80813187e-01],
        [8.78644066e-04, 5.30022731e-02],
        [7.24836808e-01, 7.29268484e-01],
        [6.42908132e-01, 8

In [9]:
U @ U.conj().T

array([[ 1.00000000e+00+0.00000000e+00j, -1.11022302e-16+3.88578059e-16j],
       [-1.11022302e-16-3.88578059e-16j,  1.00000000e+00+0.00000000e+00j]])

In [10]:
def operator_distance(A,B, distance):
    
    assert distance in ['fro','nuc'], "Distance type should be 'fro', 'nuc'."
    
    d = np.linalg.norm(A-B, ord=distance)
    return d

In [119]:
def get_normalized_Hamiltonian(U):
    '''
    U = exp(iHt), where H is Hermitian and has trace norm of 1.
    '''
    iHt = scipy.linalg.logm(U)
    t = np.linalg.norm(iHt,'nuc')
    H = -1j * iHt / t
    return H,t

In [122]:
H,t = get_normalized_Hamiltonian(Z)
recX = scipy.linalg.expm(1j*H*t)
recX

array([[ 1.+0.00000000e+00j,  0.+0.00000000e+00j],
       [ 0.+0.00000000e+00j, -1.+1.45473231e-15j]])

In [124]:
CNOT = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])
H,t = get_normalized_Hamiltonian(CNOT)
recCNOT = scipy.linalg.expm(1j*H*t)
print(H)
print(recCNOT) 


[[ 0. -0.00000000e+00j  0. -0.00000000e+00j  0. -0.00000000e+00j
   0. -0.00000000e+00j]
 [ 0. -0.00000000e+00j  0. -0.00000000e+00j  0. -0.00000000e+00j
   0. -0.00000000e+00j]
 [ 0. -0.00000000e+00j  0. -0.00000000e+00j  0.5+5.57701429e-17j
  -0.5-5.57701429e-17j]
 [ 0. -0.00000000e+00j  0. -0.00000000e+00j -0.5-5.57701429e-17j
   0.5+5.57701429e-17j]]
[[1.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j]
 [0.00000000e+00+0.00000000e+00j 1.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j]
 [0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j
  1.58271347e-16+9.07439246e-16j 1.00000000e+00-9.66165673e-16j]
 [0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j
  1.00000000e+00-9.02601973e-16j 1.44384787e-16+9.61370803e-16j]]


In [138]:
RX_pi2 = qiskit.circuit.library.RXGate(np.pi/2).to_matrix()
RX_pi3 = qiskit.circuit.library.RXGate(np.pi/3).to_matrix()

In [141]:
H_pi2 = get_normalized_Hamiltonian(RX_pi2)
H_pi3 = get_normalized_Hamiltonian(RX_pi3)
print(H_pi2)
print(H_pi3)

(array([[-7.06789929e-17+1.06018489e-16j, -5.00000000e-01-2.65046223e-17j],
       [-5.00000000e-01-1.94367231e-16j, -3.53394965e-17+9.71836153e-17j]]), 1.5707963267948961)
(array([[-5.30092447e-17+5.30092447e-17j, -5.00000000e-01+0.00000000e+00j],
       [-5.00000000e-01+7.95138670e-17j,  0.00000000e+00+1.32523112e-16j]]), 1.0471975511965974)


In [135]:
help(RX_pi2)

Help on RXGate in module qiskit.circuit.library.standard_gates.rx object:

class RXGate(qiskit.circuit.gate.Gate)
 |  RXGate(theta, label=None)
 |  
 |  Single-qubit rotation about the X axis.
 |  
 |  **Circuit symbol:**
 |  
 |  .. parsed-literal::
 |  
 |           ┌───────┐
 |      q_0: ┤ Rx(ϴ) ├
 |           └───────┘
 |  
 |  **Matrix Representation:**
 |  
 |  .. math::
 |  
 |      \newcommand{\th}{\frac{\theta}{2}}
 |  
 |      RX(\theta) = exp(-i \th X) =
 |          \begin{pmatrix}
 |              \cos{\th}   & -i\sin{\th} \\
 |              -i\sin{\th} & \cos{\th}
 |          \end{pmatrix}
 |  
 |  Method resolution order:
 |      RXGate
 |      qiskit.circuit.gate.Gate
 |      qiskit.circuit.instruction.Instruction
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __array__(self, dtype=None)
 |      Return a numpy.array for the RX gate.
 |  
 |  __init__(self, theta, label=None)
 |      Create new RX gate.
 |  
 |  control(self, num_ctrl_qubits=1, label=None,